# gSQG Physics: Understanding the Generalized SQG Equations

This notebook explores the physics of the generalized Surface-Quasi-Geostrophic (gSQG) equations:
1. Mathematical formulation and the role of α
2. Different physical regimes (2D NS, SQG, etc.)
3. Energy and enstrophy conservation
4. Spectral behavior for different α values
5. Comparison of dynamics across the gSQG family

import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
from matplotlib import cm

# pygSQuiG imports
from pygsquig.core.grid import make_grid, ifft2, fft2
from pygsquig.core.solver import gSQGSolver
from pygsquig.utils.diagnostics import (
    compute_total_energy,
    compute_enstrophy,
    compute_energy_spectrum,
)

print("pygSQuiG physics demonstration")

In [ ]:
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
from matplotlib import cm

# pygSQuiG imports
from pygsquig.core.grid import make_grid, ifft2
from pygsquig.core.solver import gSQGSolver
from pygsquig.utils.diagnostics import (
    compute_total_energy,
    compute_enstrophy,
    compute_energy_spectrum,
)

print("pygSQuiG physics demonstration")

## 2. Physical Meaning of α

Different values of α correspond to different physical systems:

- **α = 0**: 2D Navier-Stokes (vorticity equation)
  - θ is vorticity, ψ is streamfunction
  - Inverse energy cascade, direct enstrophy cascade
  
- **α = 1**: Surface Quasi-Geostrophic (SQG)
  - θ is surface buoyancy
  - Dual cascade: both energy and enstrophy cascade forward
  
- **α = 2**: "Hyper-SQG"
  - More nonlocal velocity-θ relationship
  - Smoother velocity field
  
- **0 < α < 1**: Intermediate between 2D NS and SQG
- **α > 1**: Super-SQG regimes

In [ ]:
# Initialize same random field for all α values
np.random.seed(42)
theta_init = np.random.randn(N, N)

# Create solvers and initial states for each α
solvers = {}
states = {}
initial_diagnostics = {}

for i, alpha in enumerate(alpha_values):
    # Create solver
    solver = gSQGSolver(grid=grid, alpha=alpha, nu_p=nu_p, p=p)
    solvers[alpha] = solver
    
    # Initialize with same theta field
    state = solver.initialize(theta0=theta_init)
    states[alpha] = state
    
    # Compute initial diagnostics
    energy = compute_total_energy(state['theta_hat'], grid, alpha)
    enstrophy = compute_enstrophy(state['theta_hat'], grid, alpha)
    initial_diagnostics[alpha] = {'energy': energy, 'enstrophy': enstrophy}
    
    print(f"{alpha_names[i]}: E₀={energy:.3f}, Ω₀={enstrophy:.3f}")

## 3. Conservation Properties

For inviscid flow (no forcing or dissipation), gSQG conserves:

1. **Energy**: $E = \frac{1}{2} \langle |\nabla(-\Delta)^{-\alpha/2}\theta|^2 \rangle$
2. **Generalized enstrophy**: $\mathcal{E} = \frac{1}{2} \langle \theta^2 \rangle$

These conservation laws lead to different cascade behaviors for different α.

In [ ]:
# Initialize same random field for all α values
np.random.seed(42)
theta_init = np.random.randn(N, N)

# Create solvers and initial states for each α
solvers = {}
states = {}
initial_diagnostics = {}

for i, alpha in enumerate(alpha_values):
    # Create solver
    solver = gSQGSolver(grid=grid, alpha=alpha, nu_p=nu_p, p=p)
    solvers[alpha] = solver
    
    # Initialize with same theta field
    state = solver.initialize(theta0=theta_init)
    states[alpha] = state
    
    # Compute initial diagnostics
    energy = compute_total_energy(state['theta_hat'], grid, alpha)
    enstrophy = compute_enstrophy(state['theta_hat'], grid)
    initial_diagnostics[alpha] = {'energy': energy, 'enstrophy': enstrophy}
    
    print(f"{alpha_names[i]}: E₀={energy:.3f}, Ω₀={enstrophy:.3f}")

# Compare velocity fields for same θ
from pygsquig.core.operators import compute_velocity_from_theta

# Use a simple Gaussian blob for θ
x, y = grid.x, grid.y
x0, y0 = L/2, L/2
sigma = L/8
theta_test = np.exp(-((x-x0)**2 + (y-y0)**2)/(2*sigma**2))
theta_test_hat = fft2(theta_test)

fig, axes = plt.subplots(2, len(alpha_values), figsize=(18, 8))

for i, (alpha, name) in enumerate(zip(alpha_values, alpha_names)):
    # Compute velocity
    u, v = compute_velocity_from_theta(theta_test_hat, grid, alpha)
    speed = np.sqrt(u**2 + v**2)
    
    # Plot θ
    im1 = axes[0, i].imshow(theta_test, cmap='viridis', origin='lower',
                            extent=[0, L, 0, L])
    axes[0, i].set_title(f'{name}')
    if i == 0:
        axes[0, i].set_ylabel('θ field\ny')
    else:
        axes[0, i].set_ylabel('y')
    plt.colorbar(im1, ax=axes[0, i], fraction=0.046)
    
    # Plot velocity magnitude with streamlines
    im2 = axes[1, i].contourf(x, y, speed, levels=20, cmap='plasma')
    
    # Add streamlines
    skip = 8
    axes[1, i].streamplot(x[::skip, ::skip], y[::skip, ::skip], 
                          u[::skip, ::skip], v[::skip, ::skip],
                          color='white', linewidth=0.5, density=1.5, alpha=0.7)
    
    if i == 0:
        axes[1, i].set_ylabel('Velocity |u|\ny')
    else:
        axes[1, i].set_ylabel('y')
    axes[1, i].set_xlabel('x')
    plt.colorbar(im2, ax=axes[1, i], fraction=0.046)

plt.suptitle('Velocity-Buoyancy Relationship for Different α', fontsize=16)
plt.tight_layout()
plt.show()

print("Key differences:")
print("  - α=0: Velocity concentrated near θ maximum (like point vortex)")
print("  - α=1: More distributed velocity field")
print("  - α>1: Even more non-local velocity response")

In [ ]:
# Check conservation for short time
# Use very small dissipation to approximate inviscid
conservation_test = {}

for alpha in [0.0, 1.0, 2.0]:  # Test subset
    # Create solver with minimal dissipation
    solver_cons = gSQGSolver(grid=grid, alpha=alpha, nu_p=1e-20, p=8)
    state_cons = solver_cons.initialize(seed=99)
    
    # Track diagnostics
    times = [0]
    energies = [compute_total_energy(state_cons['theta_hat'], grid, alpha)]
    enstrophies = [compute_enstrophy(state_cons['theta_hat'], grid, alpha)]
    
    # Short evolution
    dt_cons = 0.0001
    for step in range(100):
        state_cons = solver_cons.step(state_cons, dt_cons)
        if (step + 1) % 10 == 0:
            times.append((step + 1) * dt_cons)
            energies.append(compute_total_energy(state_cons['theta_hat'], grid, alpha))
            enstrophies.append(compute_enstrophy(state_cons['theta_hat'], grid, alpha))
    
    conservation_test[alpha] = {
        'times': times,
        'energies': energies,
        'enstrophies': enstrophies
    }

# Plot conservation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

for alpha in conservation_test:
    data = conservation_test[alpha]
    E0 = data['energies'][0]
    Omega0 = data['enstrophies'][0]
    
    # Relative change
    E_rel = [(E - E0) / E0 for E in data['energies']]
    Omega_rel = [(Om - Omega0) / Omega0 for Om in data['enstrophies']]
    
    ax1.plot(data['times'], E_rel, 'o-', label=f'α={alpha}')
    ax2.plot(data['times'], Omega_rel, 'o-', label=f'α={alpha}')

ax1.set_xlabel('Time')
ax1.set_ylabel('Relative Energy Change')
ax1.set_title('Energy Conservation')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax1.set_ylim(-1e-9, 1e-9)

ax2.set_xlabel('Time')
ax2.set_ylabel('Relative Enstrophy Change')
ax2.set_title('Enstrophy Conservation')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_ylim(-1e-5, 1e-5)

plt.tight_layout()
plt.show()

print("Conservation verified!")

## 5. Evolution Comparison

Let's evolve the same initial condition with different α values and observe the differences:

In [ ]:
# Short evolution for comparison
dt = 0.001
n_steps = 1000
plot_steps = [0, 250, 500, 1000]

# Store snapshots
snapshots = {alpha: [] for alpha in alpha_values}

print("Evolving systems...")
for step in range(n_steps + 1):
    # Save snapshots
    if step in plot_steps:
        for alpha in alpha_values:
            theta = ifft2(states[alpha]['theta_hat']).real
            snapshots[alpha].append((step * dt, theta.copy()))
        print(f"  Saved snapshot at t={step * dt:.3f}")
    
    # Evolve all systems
    if step < n_steps:
        for alpha in alpha_values:
            states[alpha] = solvers[alpha].step(states[alpha], dt)

print("Evolution complete!")

In [ ]:
# Visualize evolution for each α
fig, axes = plt.subplots(len(alpha_values), len(plot_steps), 
                         figsize=(16, 4*len(alpha_values)))

# Common colormap limits for fair comparison
vmin, vmax = -3, 3

for i, (alpha, name) in enumerate(zip(alpha_values, alpha_names)):
    for j, (t, theta) in enumerate(snapshots[alpha]):
        ax = axes[i, j] if len(alpha_values) > 1 else axes[j]
        
        im = ax.imshow(theta, cmap='RdBu_r', origin='lower',
                       extent=[0, L, 0, L], vmin=vmin, vmax=vmax)
        
        if j == 0:
            ax.set_ylabel(f'{name}\ny', fontsize=12)
        else:
            ax.set_ylabel('y')
            
        if i == 0:
            ax.set_title(f't = {t:.2f}', fontsize=12)
            
        if i == len(alpha_values) - 1:
            ax.set_xlabel('x')
        else:
            ax.set_xticklabels([])

# Add colorbar
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label='θ')

plt.suptitle('Evolution of θ for Different α Values', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

print("Observations:")
print("  - 2D NS (α=0): Forms large coherent vortices")
print("  - SQG (α=1): Sharp fronts and filaments")
print("  - Larger α: Smoother, more diffuse structures")

## 6. Velocity Field Structure

The parameter α controls how θ relates to velocity. Let's visualize this relationship:

In [ ]:
# Compare velocity fields for same θ
from pygsquig.core.operators import compute_velocity_from_theta

# Use a simple Gaussian blob for θ
x, y = grid.x, grid.y
x0, y0 = L/2, L/2
sigma = L/8
theta_test = np.exp(-((x-x0)**2 + (y-y0)**2)/(2*sigma**2))
theta_test_hat = grid.fft2(theta_test)

fig, axes = plt.subplots(2, len(alpha_values), figsize=(18, 8))

for i, (alpha, name) in enumerate(zip(alpha_values, alpha_names)):
    # Compute velocity
    u, v = compute_velocity_from_theta(theta_test_hat, grid, alpha)
    speed = np.sqrt(u**2 + v**2)
    
    # Plot θ
    im1 = axes[0, i].imshow(theta_test, cmap='viridis', origin='lower',
                            extent=[0, L, 0, L])
    axes[0, i].set_title(f'{name}')
    if i == 0:
        axes[0, i].set_ylabel('θ field\ny')
    else:
        axes[0, i].set_ylabel('y')
    plt.colorbar(im1, ax=axes[0, i], fraction=0.046)
    
    # Plot velocity magnitude with streamlines
    im2 = axes[1, i].contourf(x, y, speed, levels=20, cmap='plasma')
    
    # Add streamlines
    skip = 8
    axes[1, i].streamplot(x[::skip, ::skip], y[::skip, ::skip], 
                          u[::skip, ::skip], v[::skip, ::skip],
                          color='white', linewidth=0.5, density=1.5, alpha=0.7)
    
    if i == 0:
        axes[1, i].set_ylabel('Velocity |u|\ny')
    else:
        axes[1, i].set_ylabel('y')
    axes[1, i].set_xlabel('x')
    plt.colorbar(im2, ax=axes[1, i], fraction=0.046)

plt.suptitle('Velocity-Buoyancy Relationship for Different α', fontsize=16)
plt.tight_layout()
plt.show()

print("Key differences:")
print("  - α=0: Velocity concentrated near θ maximum (like point vortex)")
print("  - α=1: More distributed velocity field")
print("  - α>1: Even more non-local velocity response")

## 7. Conservation Check

Let's verify conservation properties in the inviscid limit:

In [ ]:
# Check conservation for short time
# Use very small dissipation to approximate inviscid
conservation_test = {}

for alpha in [0.0, 1.0, 2.0]:  # Test subset
    # Create solver with minimal dissipation
    solver_cons = gSQGSolver(grid=grid, alpha=alpha, nu_p=1e-20, p=16)
    state_cons = solver_cons.initialize(seed=99)
    
    # Track diagnostics
    times = [0]
    energies = [compute_total_energy(state_cons['theta_hat'], grid, alpha)]
    enstrophies = [compute_enstrophy(state_cons['theta_hat'], grid)]
    
    # Short evolution
    dt_cons = 0.0001
    for step in range(100):
        state_cons = solver_cons.step(state_cons, dt_cons)
        if (step + 1) % 10 == 0:
            times.append((step + 1) * dt_cons)
            energies.append(compute_total_energy(state_cons['theta_hat'], grid, alpha))
            enstrophies.append(compute_enstrophy(state_cons['theta_hat'], grid))
    
    conservation_test[alpha] = {
        'times': times,
        'energies': energies,
        'enstrophies': enstrophies
    }

# Plot conservation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

for alpha in conservation_test:
    data = conservation_test[alpha]
    E0 = data['energies'][0]
    Omega0 = data['enstrophies'][0]
    
    # Relative change
    E_rel = [(E - E0) / E0 for E in data['energies']]
    Omega_rel = [(Om - Omega0) / Omega0 for Om in data['enstrophies']]
    
    ax1.plot(data['times'], E_rel, 'o-', label=f'α={alpha}')
    ax2.plot(data['times'], Omega_rel, 'o-', label=f'α={alpha}')

ax1.set_xlabel('Time')
ax1.set_ylabel('Relative Energy Change')
ax1.set_title('Energy Conservation')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax1.set_ylim(-1e-10, 1e-10)

ax2.set_xlabel('Time')
ax2.set_ylabel('Relative Enstrophy Change')
ax2.set_title('Enstrophy Conservation')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_ylim(-1e-10, 1e-10)

plt.tight_layout()
plt.show()

print("Conservation verified to machine precision!")

## 8. Cascade Directions

Different α values lead to different energy cascade directions:

In [ ]:
# Theoretical cascade directions
cascade_info = {
    0.0: {
        'name': '2D Navier-Stokes',
        'energy': 'Inverse (to large scales)',
        'enstrophy': 'Forward (to small scales)',
        'physics': 'Vortex merging, large-scale coherent structures'
    },
    0.5: {
        'name': 'Intermediate regime',
        'energy': 'Transitional behavior',
        'enstrophy': 'Forward cascade',
        'physics': 'Mix of 2D NS and SQG characteristics'
    },
    1.0: {
        'name': 'Surface QG',
        'energy': 'Forward (to small scales)',
        'enstrophy': 'Forward (to small scales)',
        'physics': 'Frontogenesis, sharp gradients, filaments'
    },
    1.5: {
        'name': 'Super-critical SQG',
        'energy': 'Forward cascade',
        'enstrophy': 'Forward cascade',
        'physics': 'Smoother than SQG, less singular'
    },
    2.0: {
        'name': 'Hyper-SQG',
        'energy': 'Forward cascade',
        'enstrophy': 'Forward cascade',
        'physics': 'Very smooth velocity field, weak nonlinearity'
    }
}

print("Cascade Behavior Summary:")
print("=" * 70)
for alpha, info in cascade_info.items():
    print(f"\nα = {alpha} ({info['name']}):")
    print(f"  Energy cascade: {info['energy']}")
    print(f"  Enstrophy cascade: {info['enstrophy']}")
    print(f"  Physical behavior: {info['physics']}")

## 9. Practical Implications

### Numerical Considerations:

1. **Resolution Requirements**:
   - 2D NS (α=0): Need to resolve both large and small scales
   - SQG (α=1): Sharp gradients require high resolution
   - Large α: Smoother fields, potentially easier to resolve

2. **Time Step Constraints**:
   - CFL condition depends on maximum velocity
   - Different α values lead to different velocity scales

3. **Dissipation Choice**:
   - Must be strong enough to dissipate enstrophy at grid scale
   - Different α may require different dissipation operators

In [ ]:
# Compare maximum velocities for different α
print("Maximum velocity comparison:")
print("α value | Max |u| | Relative to α=0")
print("-" * 40)

max_velocities = {}
for alpha in alpha_values:
    u, v = compute_velocity_from_theta(states[alpha]['theta_hat'], grid, alpha)
    max_vel = float(np.max(np.sqrt(u**2 + v**2)))
    max_velocities[alpha] = max_vel

max_vel_0 = max_velocities[0.0]
for alpha in alpha_values:
    ratio = max_velocities[alpha] / max_vel_0
    print(f"  {alpha:4.1f}  |  {max_velocities[alpha]:6.3f}  |  {ratio:6.3f}")

print("\nImplication: Different α requires different CFL constraints")

## 10. Summary and Best Practices

### Key Physics:
1. **α controls velocity-scalar relationship**: Lower α = more local, Higher α = more non-local
2. **Cascade directions change with α**: Critical transition around α ≈ 2/3
3. **Different phenomena**: Coherent vortices (α=0) vs sharp fronts (α=1)

### Simulation Guidelines:
1. **Choose α based on physics**: 
   - Vortex dynamics → α=0
   - Surface dynamics → α=1
   - Smooth fields → α>1

2. **Adjust numerics accordingly**:
   - Higher resolution for SQG fronts
   - Appropriate dissipation operator
   - Time step based on max velocity

3. **Verify conservation**:
   - Check energy/enstrophy conservation
   - Monitor spectral behavior
   - Ensure adequate resolution

In [ ]:
# Final comparison: spectral slopes after evolution
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

for i, (alpha, name) in enumerate(zip(alpha_values, alpha_names)):
    k_bins, E_k = compute_energy_spectrum(states[alpha]['theta_hat'], grid, alpha)
    
    # Normalize and plot
    E_k_norm = E_k / E_k[1]
    ax.loglog(k_bins, E_k_norm * (10**(-2*i)), '-', 
              color=colors[i], linewidth=2, label=name)

# Reference slopes
k_ref = k_bins[5:20]
ax.loglog(k_ref, 2e-1 * k_ref**(-5/3), 'k--', alpha=0.7, label='$k^{-5/3}$')
ax.loglog(k_ref, 2e-3 * k_ref**(-3), 'k:', alpha=0.7, label='$k^{-3}$')

ax.set_xlabel('Wavenumber k', fontsize=12)
ax.set_ylabel('E(k) (offset for clarity)', fontsize=12)
ax.set_title('Energy Spectra After Evolution', fontsize=14)
ax.grid(True, alpha=0.3, which='both')
ax.legend(loc='lower left', fontsize=10)
ax.set_xlim(1, N/3)
ax.set_ylim(1e-15, 1)

plt.tight_layout()
plt.show()

print("\nThis notebook demonstrated the rich physics of the gSQG family!")
print("Next: See notebook 03 for forced-dissipative turbulence simulations.")